In [ ]:
# AutoGit2

分类总结。

In [1]:
import pygit2,os
from pygit2 import discover_repository,Repository

## Repo

### git-init

Creating a new repository

Create bare repository

`$ git init --bare path/to/git`

````
>>> pygit2.init_repository('path/to/git', True)
<pygit2.repository.Repository object at 0x10f08b680>
Create standard repository
````

`$ git init path/to/git`

````
>>> pygit2.init_repository('path/to/git', False)
<pygit2.repository.Repository object at 0x10f08b680>
````

### 查找并打开repo

In [2]:
current_working_directory = os.getcwd()
repository_path = discover_repository(current_working_directory)
repo = Repository(repository_path)

## 引用Reference、Oid、commit的转化

`class pygit2.Repository(path=None, flags=0).resolve_refish(refish)`

Convert a reference-like short name “ref-ish” to a valid (commit, reference) pair.

If ref-ish points to a commit, the reference element of the result will be None.

Examples:
````python
repo.resolve_refish('mybranch')
repo.resolve_refish('sometag')
repo.resolve_refish('origin/master')
repo.resolve_refish('bbb78a9')
````

In [18]:
print('type of object:')
print('repo.head:\n\t{}'.format(type(repo.head)))
print('repo.head.target:\n\t{}'.format(type(repo.head.target)))
print('repo.head.resolve():\n\t{}'.format(type(repo.head.resolve())))
t= repo.resolve_refish('HEAD')
print('repo.resolve_refish("HEAD"):\n\t{}\n\t=={}'.format(type(t),(t[0],type(t[1]))))
print('repo[repo.head.target]:\n\t{}'.format(repo[repo.head.target]))
print('repo.head.peel():\n\t{}'.format(repo.head.peel()))

type of object:
repo.head:
	<class '_pygit2.Reference'>
repo.head.target:
	<class '_pygit2.Oid'>
repo.head.resolve():
	<class '_pygit2.Reference'>
repo.resolve_refish("HEAD"):
	<class 'tuple'>
	==(<pygit2.Object{commit:a1f4fdbda3bcec69430d81892f689444d8d41b8a}>, <class '_pygit2.Reference'>)
repo[repo.head.target]:
	<pygit2.Object{commit:a1f4fdbda3bcec69430d81892f689444d8d41b8a}>
repo.head.peel():
	<pygit2.Object{commit:a1f4fdbda3bcec69430d81892f689444d8d41b8a}>


## commit

### create commit



Creating commits

````python
class pygit2.Repository(path=None, flags=0)
    create_commit(reference_name, author, committer, message, tree, parents[, encoding]) → Oid

````

Create a new commit object, return its oid.

Commits can be created by calling the create_commit method of the repository with the following parameters:






（在官方文档中）

In [ ]:

author = Signature('Alice Author', 'alice@authors.tld')
committer = Signature('Cecil Committer', 'cecil@committers.tld')
tree = repo.TreeBuilder().write()
repo.create_commit(
'refs/heads/master', # the name of the reference to update
author, committer, 'one line commit message\n\ndetailed commit message',
tree, # binary string representing the tree object ID
[] # list of binary strings representing parents of the new commit
)
'#\xe4<u\xfe\xd6\x17\xa0\xe6\xa2\x8b\xb6\xdc35$\xcf-\x8b~'

（我实际使用的）

In [ ]:
def mycommit(repo,msg,is_first=False):
    index = repo.index
    status = repo.status()
    for filepath, flag in status.items():
        if flag == GIT_STATUS_WT_DELETED:
            msg = 'Remove file %s' % filepath
            del index[filepath]
            
        elif flag == GIT_STATUS_WT_NEW:
            msg = 'Add file %s' % filepath
            index.add(filepath)
            
        elif flag == GIT_STATUS_WT_MODIFIED:
            msg = 'Change file %s' % filepath
            index.add(filepath)
       # else:
            #raise Exception([filepath,statusFlag(flag)])
    cmt = docommit(repo, index, msg,is_first)
    return cmt

def docommit(repo, index, msg,is_first=False):
    index.write()
    tree = index.write_tree()
    committer = Signature('gitsync', 'root@localhost')
    ref = 'HEAD' 
    parents = [] if is_first else [repo.head.target]
    sha = repo.create_commit(
        ref, committer, committer, msg, tree, parents)
    commit = repo[sha]
    return commit

### git-log

Showing HEAD commit logs

Show HEAD commit

`$ git log -1`

````
>>> commit = repo[repo.head.target]
>>> commit.message
'commit message'
````

Traverse commit history

`$ git log`

````
>>> last = repo[repo.head.target]
>>> for commit in repo.walk(last.id, pygit2.GIT_SORT_TIME):
>>>     print(commit.message) # or some other operation
````

In [29]:
cmt_c = repo.head.peel()
print('Attrs of Commit:')
s = dir(cmt_c)
s = [name for name in s if name[:2]!='__']
print('\t',s)
print('HEAD commit:')
for name in s:
    if name in ('author','committer'):
        v = getattr(cmt_c,name)
        print('\t{}({}):'.format(name,type(v)))
        print('\t\t{}:{}'.format('email',v.email))
        print('\t\t{}:{}'.format('name',v.name))
    else:
        print('\t{}:{}'.format(name,getattr(cmt_c,name)))

Attrs of Commit:
	 ['_pointer', 'author', 'commit_time', 'commit_time_offset', 'committer', 'filemode', 'gpg_signature', 'hex', 'id', 'message', 'message_encoding', 'name', 'oid', 'parent_ids', 'parents', 'peel', 'raw_message', 'raw_name', 'read_raw', 'short_id', 'tree', 'tree_id', 'type', 'type_str']
HEAD commit:
	_pointer:b'\x10t\x1c\xc5\x9d\x01\x00\x00'
	author(<class '_pygit2.Signature'>):
		email:root@localhost
		name:gitsync
	commit_time:1612277217
	commit_time_offset:480
	committer(<class '_pygit2.Signature'>):
		email:root@localhost
		name:gitsync
	filemode:None
	gpg_signature:(None, None)
	hex:a1f4fdbda3bcec69430d81892f689444d8d41b8a
	id:a1f4fdbda3bcec69430d81892f689444d8d41b8a
	message:2nd
	message_encoding:None
	name:None
	oid:a1f4fdbda3bcec69430d81892f689444d8d41b8a
	parent_ids:[ac68f819e035ac4ae2a17a8dc046d59d77f58e55]
	parents:[<pygit2.Object{commit:ac68f819e035ac4ae2a17a8dc046d59d77f58e55}>]
	peel:<built-in method peel of _pygit2.Commit object at 0x0000019DC8B5D4E0>
	raw

## Reference


````
class pygit2.Repository(path=None, flags=0)
    create_reference(name, target, force=False, message=None)
````

Create a new reference “name” which points to an object or to another reference.

Based on the type and value of the target parameter, this method tries to guess whether it is a direct or a symbolic reference.

Keyword arguments:

`force: bool`

If True references will be overridden, otherwise (the default) an exception is raised.

`message: str`

Optional message to use for the reflog.

Examples:
````python
repo.create_reference('refs/heads/foo', repo.head.target)
repo.create_reference('refs/tags/foo', 'refs/heads/master')
repo.create_reference('refs/tags/foo', 'bbb78a9cec580') # i.e. create a tag
````

In [8]:
print(repo.head.name)
all_refs = list(repo.references)
print(all_refs)

refs/heads/master
['refs/heads/master', 'refs/heads/new-branch', 'refs/remotes/origin/master']


### git-tag

git-tag

Showing all tags

`$ git tag`

````
import re
regex = re.compile('^refs/tags/')
[r for r in repo.references if regex.match(r)]
````

### Equivalents

In [53]:
print(repo.head.target==repo.resolve_refish('HEAD')[1].target)
print(repo.head.target == repo.references['HEAD'].resolve().target)
print(repo.references.get('refs/heads/master').target==repo.references['refs/heads/master'].target)

True
True
True


## Branches

In [45]:
# Listing all branches
branches_list = list(repo.branches)
# Local only
local_branches = list(repo.branches.local)
# Remote only
remote_branches = list(repo.branches.remote)


print(branches_list)
print(local_branches)
print(remote_branches)

['master', 'new-branch']
['master', 'new-branch']
[]


In [39]:

# Get a branch
branch = repo.branches['master']
other_branch = repo.branches.get('does-not-exist')  # Returns None

print(branch)
print(other_branch)
print('attrs of Branch:')
s = dir(branch)
s= [name for name in s if name[:2]!='__']
print('\t',s)
for name in s:
    try:
        print('\t{}:{}'.format(name,getattr(branch,name)))
    except Exception as e:
        print('\t{}(Exception)({}):{}'.format(name,type(e),e))



None
attrs of Branch:
	 ['branch_name', 'delete', 'is_checked_out', 'is_head', 'log', 'name', 'peel', 'raw_branch_name', 'raw_name', 'raw_shorthand', 'raw_target', 'remote_name', 'rename', 'resolve', 'set_target', 'shorthand', 'target', 'type', 'upstream', 'upstream_name']
	branch_name:master
	delete:<built-in method delete of _pygit2.Branch object at 0x0000019DC8CCC9B0>
	is_checked_out:<built-in method is_checked_out of _pygit2.Branch object at 0x0000019DC8CCC9B0>
	is_head:<built-in method is_head of _pygit2.Branch object at 0x0000019DC8CCC9B0>
	log:<built-in method log of _pygit2.Branch object at 0x0000019DC8CCC9B0>
	name:refs/heads/master
	peel:<built-in method peel of _pygit2.Branch object at 0x0000019DC8CCC9B0>
	raw_branch_name:b'master'
	raw_name:b'refs/heads/master'
	raw_shorthand:b'master'
	raw_target:a1f4fdbda3bcec69430d81892f689444d8d41b8a
	remote_name(Exception)(<class 'ValueError'>):reference 'refs/heads/master' is not a remote branch.
	rename:<built-in method rename of _py

In [36]:
other_branch = repo.branches['does-not-exist']  # Will raise a KeyError

KeyError: 'Branch not found: does-not-exist'

In [ ]:
# remote_branch = repo.branches.remote['upstream/feature']


In [44]:

# Create a local branch
new_branch = repo.branches.local.create('new-branch',repo.head.peel())

print('branches:')
print(list(repo.branches))




branches:
['master', 'new-branch']


In [43]:
# And delete it
repo.branches.delete('new-branch')

print('branches:')
print(list(repo.branches))

branches:
['master']


## Index



### git-add / git-reset

Add file contents to the index / Stage

We can add a new (untracked) file or a modified file to the index.

`$ git add foo.txt`

````
>>> index = repo.index
>>> index.add(path)
>>> index.write()
````

Restore the entry in the index / Unstage

`$ git reset HEAD src/tree.c`

````
>>> index = repo.index

# Remove path from the index
>>> path = 'src/tree.c'
>>> index.remove(path)

# Restore object from db
>>> obj = repo.revparse_single('HEAD').tree[path] # Get object from db
>>> index.add(pygit2.IndexEntry(path, obj.oid, obj.filemode)) # Add to index

# Write index
>>> index.write()
````

Query the index state / Is file staged ?

`$ git status foo.txt`

````
# Return True is the file is modified in the working tree
>>> repo.status_file(path) & pygit2.GIT_STATUS_WT_MODIFIED
````

## Diff


In [5]:
d = repo.diff('HEAD')
l = list(d)
print(l[0].text)

diff --git a/AutoGit2.ipynb b/AutoGit2.ipynb
index 1ccf8f2..2a8f9b8 100644
--- a/AutoGit2.ipynb
+++ b/AutoGit2.ipynb
@@ -35,7 +35,7 @@
   },
   {
    "cell_type": "code",
-   "execution_count": 4,
+   "execution_count": 1,
    "metadata": {},
    "outputs": [],
    "source": [
@@ -85,7 +85,7 @@
   },
   {
    "cell_type": "code",
-   "execution_count": null,
+   "execution_count": 2,
    "metadata": {},
    "outputs": [],
    "source": [
@@ -152,14 +152,12 @@
    "metadata": {}
   },
   {
-   "cell_type": "code",
-   "execution_count": null,
-   "metadata": {},
-   "outputs": [],
    "source": [
     "### create commit\n",
     "\n"
-   ]
+   ],
+   "cell_type": "markdown",
+   "metadata": {}
   },
   {
    "source": [
@@ -611,6 +609,39 @@
    ],
    "cell_type": "markdown",
    "metadata": {}
+  },
+  {
+   "source": [
+    "## Diff\n"
+   ],
+   "cell_type": "markdown",
+   "metadata": {}
+  },
+  {
+   "cell_type": "code",
+   "execution_count": 4,
+   "metadata": {},
+   "outputs":

'refs/heads/master'

## Status


In [4]:
s = 'GIT_STATUS_CURRENT,GIT_STATUS_INDEX_NEW,GIT_STATUS_INDEX_MODIFIED,GIT_STATUS_INDEX_DELETED,GIT_STATUS_INDEX_RENAMED,GIT_STATUS_INDEX_TYPECHANGE,GIT_STATUS_WT_NEW,GIT_STATUS_WT_MODIFIED,GIT_STATUS_WT_DELETED,GIT_STATUS_WT_TYPECHANGE,GIT_STATUS_WT_RENAMED,GIT_STATUS_WT_UNREADABLE,GIT_STATUS_IGNORED,GIT_STATUS_CONFLICTED'
names = s.split(',')
from pygit2 import GIT_STATUS_CURRENT,GIT_STATUS_INDEX_NEW,GIT_STATUS_INDEX_MODIFIED,GIT_STATUS_INDEX_DELETED,GIT_STATUS_INDEX_RENAMED,GIT_STATUS_INDEX_TYPECHANGE,GIT_STATUS_WT_NEW,GIT_STATUS_WT_MODIFIED,GIT_STATUS_WT_DELETED,GIT_STATUS_WT_TYPECHANGE,GIT_STATUS_WT_RENAMED,GIT_STATUS_WT_UNREADABLE,GIT_STATUS_IGNORED,GIT_STATUS_CONFLICTED

values = [GIT_STATUS_CURRENT,GIT_STATUS_INDEX_NEW,GIT_STATUS_INDEX_MODIFIED,GIT_STATUS_INDEX_DELETED,GIT_STATUS_INDEX_RENAMED,GIT_STATUS_INDEX_TYPECHANGE,GIT_STATUS_WT_NEW,GIT_STATUS_WT_MODIFIED,GIT_STATUS_WT_DELETED,GIT_STATUS_WT_TYPECHANGE,GIT_STATUS_WT_RENAMED,GIT_STATUS_WT_UNREADABLE,GIT_STATUS_IGNORED,GIT_STATUS_CONFLICTED]

for n,v in zip(names,values):
    print('{}={}'.format(n,v))


def statusFlag(flag_int):
    s = 'GIT_STATUS_CURRENT,GIT_STATUS_INDEX_NEW,GIT_STATUS_INDEX_MODIFIED,GIT_STATUS_INDEX_DELETED,GIT_STATUS_INDEX_RENAMED,GIT_STATUS_INDEX_TYPECHANGE,GIT_STATUS_WT_NEW,GIT_STATUS_WT_MODIFIED,GIT_STATUS_WT_DELETED,GIT_STATUS_WT_TYPECHANGE,GIT_STATUS_WT_RENAMED,GIT_STATUS_WT_UNREADABLE,GIT_STATUS_IGNORED,GIT_STATUS_CONFLICTED'
    names = s.split(',')

    values = [GIT_STATUS_CURRENT,GIT_STATUS_INDEX_NEW,GIT_STATUS_INDEX_MODIFIED,GIT_STATUS_INDEX_DELETED,GIT_STATUS_INDEX_RENAMED,GIT_STATUS_INDEX_TYPECHANGE,GIT_STATUS_WT_NEW,GIT_STATUS_WT_MODIFIED,GIT_STATUS_WT_DELETED,GIT_STATUS_WT_TYPECHANGE,GIT_STATUS_WT_RENAMED,GIT_STATUS_WT_UNREADABLE,GIT_STATUS_IGNORED,GIT_STATUS_CONFLICTED]

    iii = None
    result = []
    for i,v in enumerate(values):
        if v&flag_int!=0:
            result.append(names[i])
    return result




GIT_STATUS_CURRENT=0
GIT_STATUS_INDEX_NEW=1
GIT_STATUS_INDEX_MODIFIED=2
GIT_STATUS_INDEX_DELETED=4
GIT_STATUS_INDEX_RENAMED=8
GIT_STATUS_INDEX_TYPECHANGE=16
GIT_STATUS_WT_NEW=128
GIT_STATUS_WT_MODIFIED=256
GIT_STATUS_WT_DELETED=512
GIT_STATUS_WT_TYPECHANGE=1024
GIT_STATUS_WT_RENAMED=2048
GIT_STATUS_WT_UNREADABLE=4096
GIT_STATUS_IGNORED=16384
GIT_STATUS_CONFLICTED=32768


In [3]:
from pygit2 import GIT_STATUS_CURRENT,GIT_STATUS_IGNORED
status = repo.status()
for filepath, flags in status.items():
    if flags != GIT_STATUS_CURRENT:
        print("Filepath %s isn't clean" % filepath)

Filepath AutoGit2.ipynb isn't clean


In [5]:
status = repo.status()
for filepath, flag in status.items():
    print('{}:{}({})'.format(filepath,statusFlag(flag),flag))

AutoGit2.ipynb:['GIT_STATUS_WT_MODIFIED'](256)
